In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from pandas import DataFrame as DF
import xgboost as xgb
import lightgbm as lgb
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier as GBC

In [2]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.cross_validation import cross_val_score as cv
train = pd.read_csv('f_train_20180204.csv',encoding='gbk')
test = pd.read_csv('f_test_a_20180204.csv',encoding='gbk')

In [3]:
del train['id']
del test['id']
feature_name = [i for i in train.columns if i!='label']

In [4]:
def get_model(nums,cv_fold):
    feature_name1 = train_data[feature_name].columns
    get_ans_face = list(set(get_pic(gbc_model,feature_name1).head(nums)['name'])&set(get_pic(xgb_model,feature_name1).head(nums)['name'])&set(get_pic(lgb_model,feature_name1).head(nums)['name']))
    print('New Feature: ',len(get_ans_face))
    if 'SNP32*SNP34' not in get_ans_face:
        get_ans_face.append('SNP32*SNP34')
    print('New Feature: ',len(get_ans_face))
    new_lgb_model = lgb.LGBMClassifier(objective='binary',n_estimators=300,max_depth=3,min_child_samples=6,learning_rate=0.102,random_state=1)
    cv_model = cv(new_lgb_model, train_data[get_ans_face], train_label,  cv=cv_fold, scoring='f1')
    new_lgb_model.fit(train_data[get_ans_face], train_label)
    m1 = cv_model.mean()

    new_xgb_model1 = xgb.XGBClassifier(objective='binary:logistic',n_estimators=300,max_depth=4,learning_rate=0.101,random_state=1)
    cv_model = cv(new_xgb_model1, train_data[get_ans_face].values, train_label,  cv=cv_fold, scoring='f1')
    new_xgb_model1.fit(train_data[get_ans_face].values, train_label)
    m2 = cv_model.mean()

    new_gbc_model = GBC(n_estimators=310,subsample=1,min_samples_split=2,max_depth=3,learning_rate=0.1900,min_weight_fraction_leaf=0.1)
    kkk = train_data[get_ans_face].fillna(7)
    cv_model = cv(new_gbc_model, kkk[get_ans_face], train_label,  cv=cv_fold, scoring='f1')
    new_gbc_model.fit(kkk.fillna(7),train_label)

    m3 = cv_model.mean()
    print((m1+m2+m3)/3)
    pro1 = new_lgb_model.predict_proba(test_data[get_ans_face])
    pro2 = new_xgb_model1.predict_proba(test_data[get_ans_face].values)
    pro3 = new_gbc_model.predict_proba(test_data[get_ans_face].fillna(7).values)
    ans = (pro1+pro2+pro3)/3
    return ans
    
# temp = [140,160,180,200,220,240,260,280,300,320]

# ans = []
# for i in range(len(temp)):
#     print('Now All Feature:',temp[i])
#     ans = get_model(temp[i],5)
#     if i == 0:
#         ans1 = ans
#     else:
#         ans1 += ans
# ans1 /= len(temp)

def find_best_feature(feature_name,cv_fold):
    get_ans_face = feature_name
    new_lgb_model = lgb.LGBMClassifier(objective='binary',n_estimators=300,max_depth=3,min_child_samples=6,learning_rate=0.102,random_state=1)
    cv_model = cv(new_lgb_model, train_data[get_ans_face], train_label,  cv=cv_fold, scoring='f1')
    new_lgb_model.fit(train_data[get_ans_face], train_label)
    m1 = cv_model.mean()

    new_xgb_model1 = xgb.XGBClassifier(objective='binary:logistic',n_estimators=300,max_depth=4,learning_rate=0.101,random_state=1)
    cv_model = cv(new_xgb_model1, train_data[get_ans_face].values, train_label,  cv=cv_fold, scoring='f1')
    new_xgb_model1.fit(train_data[get_ans_face].values, train_label)
    m2 = cv_model.mean()

    new_gbc_model = GBC(n_estimators=310,subsample=1,min_samples_split=2,max_depth=3,learning_rate=0.1900,min_weight_fraction_leaf=0.1)
    kkk = train_data[get_ans_face].fillna(7)
    cv_model = cv(new_gbc_model, kkk[get_ans_face], train_label,  cv=cv_fold, scoring='f1')
    new_gbc_model.fit(kkk.fillna(7),train_label)
    m3 = cv_model.mean()
    return (m1+m2+m3)/3

def train_best_feature(feature_name):
    get_ans_face = feature_name
    new_lgb_model = lgb.LGBMClassifier(objective='binary',n_estimators=300,max_depth=3,min_child_samples=6,learning_rate=0.102,random_state=1)
    new_lgb_model.fit(train_data[get_ans_face], train_label)

    new_xgb_model1 = xgb.XGBClassifier(objective='binary:logistic',n_estimators=300,max_depth=4,learning_rate=0.101,random_state=1)
    new_xgb_model1.fit(train_data[get_ans_face].values, train_label)

    new_gbc_model = GBC(n_estimators=310,subsample=1,min_samples_split=2,max_depth=3,learning_rate=0.1900,min_weight_fraction_leaf=0.1)
    kkk = train_data[get_ans_face].fillna(7)
    new_gbc_model.fit(kkk.fillna(7),train_label)
    
    pro1 = new_lgb_model.predict_proba(test_data[get_ans_face])
    pro2 = new_xgb_model1.predict_proba(test_data[get_ans_face].values)
    pro3 = new_gbc_model.predict_proba(test_data[get_ans_face].fillna(7).values)
    ans = (pro1+pro2+pro3)/3

    return ans

In [5]:
train_data = pd.concat([train],axis=0)
train_label = train_data['label']
del train_data['label']
test_data = test[feature_name]

In [6]:
feature_SNP = [i for i in feature_name if 'SNP' in i]
feature_no_SNP = list(set(feature_name)-set(feature_SNP))
train_no_SNP_mean = train.describe().T[['mean','min','max']].T[feature_no_SNP]
train_no_SNP = train[feature_no_SNP]
train_SNP = train[feature_SNP]
test_no_SNP_mean = test.describe().T[['mean','min','max']].T[feature_no_SNP]
test_SNP = test[feature_SNP]
test_no_SNP = test[feature_no_SNP]

In [7]:
train_no_SNP.to_csv('train_no_SNP.csv',index=False)
test_no_SNP.to_csv('test_no_SNP.csv',index=False)
train_SNP.to_csv('train_SNP.csv',index=False)
test_SNP.to_csv('test_SNP.csv',index=False)

In [8]:
def get_division_feature(data,feature_name):
    new_feature = []
    new_feature_name = []
    for i in range(len(data[feature_name].columns)-1):
        for j in range(i+1,len(data[feature_name].columns)):
            new_feature_name.append(data[feature_name].columns[i] + '/' + data[feature_name].columns[j])
            new_feature_name.append(data[feature_name].columns[i] + '*' + data[feature_name].columns[j])
            new_feature_name.append(data[feature_name].columns[i] + '+' + data[feature_name].columns[j])
            new_feature_name.append(data[feature_name].columns[i] + '-' + data[feature_name].columns[j])
            new_feature.append(data[data[feature_name].columns[i]]/data[data[feature_name].columns[j]])
            new_feature.append(data[data[feature_name].columns[i]]*data[data[feature_name].columns[j]])
            new_feature.append(data[data[feature_name].columns[i]]+data[data[feature_name].columns[j]])
            new_feature.append(data[data[feature_name].columns[i]]-data[data[feature_name].columns[j]])
            
    
    temp_data = DF(pd.concat(new_feature,axis=1))
    temp_data.columns = new_feature_name
    data = pd.concat([data,temp_data],axis=1).reset_index(drop=True)
    
    print(data.shape)
    
    return data.reset_index(drop=True)

def get_square_feature(data,feature_name):
    new_feature = []
    new_feature_name = []
    for i in range(len(data[feature_name].columns)):
        new_feature_name.append(data[feature_name].columns[i] + '**2')
        new_feature_name.append(data[feature_name].columns[i] + '**1/2')
        new_feature.append(data[data[feature_name].columns[i]]**2)
        new_feature.append(data[data[feature_name].columns[i]]**(1/2))
        
    temp_data = DF(pd.concat(new_feature,axis=1))
    temp_data.columns = new_feature_name
    data = pd.concat([data,temp_data],axis=1).reset_index(drop=True)
    
    print(data.shape)
    
    return data.reset_index(drop=True)

In [9]:
train_data = get_square_feature(train_no_SNP,feature_no_SNP)
test_data = get_square_feature(test_no_SNP,feature_no_SNP)

train_data_SNP = get_division_feature(train_SNP,train_SNP.columns)
train_data_no_SNP = get_division_feature(train_no_SNP,train_no_SNP.columns)
train_data = pd.concat([train_data_SNP,train_data_no_SNP,train_data],axis=1)
test_data_SNP = get_division_feature(test_SNP,test_SNP.columns)
test_data_no_SNP = get_division_feature(test_no_SNP,test_no_SNP.columns)
test_data = pd.concat([test_data_SNP,test_data_no_SNP,test_data],axis=1)

feature_name = [i for i in train_data.columns if i!='label']
print(len(train_data.columns))
print(train_data.shape)
print(test_data.shape)

(1000, 56)
(200, 56)
(1000, 5995)
(1000, 1540)
(200, 5995)
(200, 1540)
7591
(1000, 7591)
(200, 7591)


In [10]:
lgb_model = lgb.LGBMClassifier(objective='binary',n_estimators=120,subsample=0.9,nthread=4)
# cv_model = cv(lgb_model, train_data[feature_name], train_label,  cv=10, scoring='f1')
lgb_model.fit(train_data[feature_name], train_label)
# print(cv_model)
# print(cv_model.mean())

# mean 0.650 166 feature
# mean 0.650 6900 feature
# median 0.648

LGBMClassifier(boosting_type='gbdt', colsample_bytree=1.0, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=120,
        n_jobs=-1, nthread=4, num_leaves=31, objective='binary',
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=0.9, subsample_for_bin=200000, subsample_freq=1)

In [11]:
def get_pic(model,feature_name):
    ans = DF()
    ans['name'] = feature_name
    ans['score'] = model.feature_importances_
#     print(ans[ans['score']>0].shape)
    return ans.sort_values(by=['score'],ascending=False).reset_index(drop=True)

In [12]:
xgb_model = xgb.XGBClassifier(objective='binary:logistic',n_estimators=120,subsample=0.9,nthread=4)
# cv_model = cv(xgb_model, train_data[feature_name].values, train_label,  cv=10, scoring='f1')
xgb_model.fit(train_data[feature_name].values, train_label)
# print(cv_model)
# print(cv_model.mean())

# mean 166 632
# median 0.657

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=120,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.9)

In [13]:
gbc_model = GBC(n_estimators=200,subsample=0.9,min_samples_split=2)
kkk = train_data[feature_name].fillna(7)
kkk.replace(np.inf,999,inplace=True)
# cv_model = cv(gbc_model, kkk[feature_name], train_label,  1cv=10, scoring='f1')
gbc_model.fit(kkk.fillna(7),train_label)
# print(cv_model)
# print(cv_model.mean())

# mean 0.653
# median 0.664

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200,
              presort='auto', random_state=None, subsample=0.9, verbose=0,
              warm_start=False)

In [14]:
len(feature_name)

7591

In [21]:
nums = 45
feature_name1 = train_data[feature_name].columns
get_ans_face = list(set(get_pic(lgb_model,feature_name1).head(nums)['name'])|set(get_pic(xgb_model,feature_name1).head(nums)['name'])|set(get_pic(gbc_model,feature_name1).head(nums)['name']))
print('New Feature: ',len(get_ans_face))

# 320 0.739

New Feature:  96


In [22]:
now_feature = []
check = 0
for i in range(len(get_ans_face)):
    now_feature.append(get_ans_face[i])
    jj = find_best_feature(now_feature,6)
    if jj>check:
        print('目前特征长度为',len(now_feature),' 目前帅气的cv值是',jj,' 成功加入第',i+1,'个','增值为',jj-check)
        check = jj
    else:
        now_feature.pop()
#         print('目前特征长度为',len(now_feature),'第',i+1,'个拉闸了')

目前特征长度为 1  目前帅气的cv值是 0.386775127677  成功加入第 1 个 增值为 0.386775127677
目前特征长度为 2  目前帅气的cv值是 0.51616384038  成功加入第 2 个 增值为 0.129388712703
目前特征长度为 3  目前帅气的cv值是 0.527535265985  成功加入第 3 个 增值为 0.011371425605
目前特征长度为 4  目前帅气的cv值是 0.563174983085  成功加入第 4 个 增值为 0.0356397171
目前特征长度为 5  目前帅气的cv值是 0.57436190063  成功加入第 5 个 增值为 0.0111869175454
目前特征长度为 6  目前帅气的cv值是 0.586587422568  成功加入第 7 个 增值为 0.0122255219373
目前特征长度为 7  目前帅气的cv值是 0.593785226558  成功加入第 12 个 增值为 0.00719780399015
目前特征长度为 8  目前帅气的cv值是 0.608606465091  成功加入第 14 个 增值为 0.0148212385332
目前特征长度为 9  目前帅气的cv值是 0.609209748232  成功加入第 17 个 增值为 0.000603283141013
目前特征长度为 10  目前帅气的cv值是 0.620925798111  成功加入第 18 个 增值为 0.011716049879
目前特征长度为 11  目前帅气的cv值是 0.634570115268  成功加入第 19 个 增值为 0.0136443171573
目前特征长度为 12  目前帅气的cv值是 0.688309863978  成功加入第 20 个 增值为 0.0537397487097
目前特征长度为 13  目前帅气的cv值是 0.689758693609  成功加入第 21 个 增值为 0.00144882963117
目前特征长度为 14  目前帅气的cv值是 0.692031700018  成功加入第 22 个 增值为 0.00227300640844
目前特征长度为 15  目前帅气的cv值是 0.70464125809  成功加入第 24 个 增值为 0

In [17]:
now_feature

['VAR00007*DM家族史',
 'AST-hsCRP',
 '分娩时/wbc',
 'wbc-LDLC',
 'SNP32*SNP33',
 'SNP20*SNP34',
 'ApoB/BUN',
 'SNP37/SNP53',
 'SNP22/SNP34',
 'VAR00007',
 'hsCRP+年龄',
 '年龄+LDLC',
 'VAR00007*年龄',
 'SNP39*SNP47',
 'hsCRP-LDLC',
 'TG*年龄',
 '孕次/Lpa',
 'SNP46/SNP47',
 'SNP26*SNP48',
 'wbc-年龄',
 '孕前BMI/Cr',
 'VAR00007*糖筛孕周',
 'SNP16/SNP34',
 '舒张压/ApoA1',
 'BUN/DM家族史',
 '孕前体重-RBP4',
 'SNP45*SNP46',
 'SNP36*SNP49',
 'SNP11*SNP15',
 'SNP33/SNP46',
 'TG+wbc',
 'HDLC/wbc',
 'TG*ALT']

First 1

In [23]:
now_feature2 = []
check = 0
for i in range(len(get_ans_face)):
    now_feature2.append(get_ans_face[len(get_ans_face)-i-1])
    jj = find_best_feature(now_feature2,6)
    if jj>check:
        print('目前特征长度为',len(now_feature2),' 目前帅气的cv值是',jj,' 成功加入第',i+1,'个','增值为',jj-check)
        check = jj
    else:
        now_feature2.pop()
#         print('目前特征长度为',len(now_feature),'第',i+1,'个拉闸了')

目前特征长度为 1  目前帅气的cv值是 0.529126383031  成功加入第 1 个 增值为 0.529126383031
目前特征长度为 2  目前帅气的cv值是 0.541969584999  成功加入第 2 个 增值为 0.0128432019677
目前特征长度为 3  目前帅气的cv值是 0.568554327993  成功加入第 3 个 增值为 0.0265847429934
目前特征长度为 4  目前帅气的cv值是 0.57652332479  成功加入第 11 个 增值为 0.0079689967979
目前特征长度为 5  目前帅气的cv值是 0.59432805946  成功加入第 12 个 增值为 0.0178047346692
目前特征长度为 6  目前帅气的cv值是 0.594995772882  成功加入第 14 个 增值为 0.000667713422274
目前特征长度为 7  目前帅气的cv值是 0.601384057634  成功加入第 18 个 增值为 0.00638828475201
目前特征长度为 8  目前帅气的cv值是 0.621135701011  成功加入第 20 个 增值为 0.0197516433766
目前特征长度为 9  目前帅气的cv值是 0.659833147249  成功加入第 24 个 增值为 0.0386974462387
目前特征长度为 10  目前帅气的cv值是 0.678642746028  成功加入第 25 个 增值为 0.0188095987783
目前特征长度为 11  目前帅气的cv值是 0.685003138629  成功加入第 31 个 增值为 0.00636039260157
目前特征长度为 12  目前帅气的cv值是 0.686918440568  成功加入第 33 个 增值为 0.00191530193904
目前特征长度为 13  目前帅气的cv值是 0.689605039799  成功加入第 44 个 增值为 0.00268659923099
目前特征长度为 14  目前帅气的cv值是 0.691387941235  成功加入第 53 个 增值为 0.00178290143601
目前特征长度为 15  目前帅气的cv值是 0.699233952221  成功加入

In [ ]:
def get_proba(ans):
    kfc = []
    tot0 = 0
    tot1 = 0
    for i in range(len(ans)):
        if ans[i][0]>0.5:
            kfc.append(0)
            tot0 += 1
        else:
            kfc.append(1)
            tot1 += 1
    print('1 = ',tot1,' ','0 =',tot0)
    return kfc
# ans1 = get_proba(train_best_feature(now_feature_1))
ans1 = get_proba(train_best_feature(now_feature))
# ans3 = get_proba((train_best_feature(now_feature2)+train_best_feature(now_feature))/2)
ans2 = get_proba(train_best_feature(now_feature2))

In [409]:
DF(ans).to_csv('真的不想做了.csv',header=False,index=False)

In [ ]:
pro1 = lgb_model.predict_proba(test_data[feature_name])
pro2 = xgb_model.predict_proba(test_data[feature_name].values)
pro3 = gbc_model.predict_proba(test_data[feature_name].fillna(7).values)

In [ ]:
lgb_ans = lgb_model.predict(test_data[feature_name])
xgb_ans = xgb_model.predict(test_data[feature_name].values)
gbc_ans = gbc_model.predict(test_data[feature_name].fillna(7.01))

In [115]:
kfc = []
for i in range(len(lgb_ans)):
    if (lgb_ans[i]==xgb_ans[i]):
        kfc.append(lgb_ans[i])
    elif (lgb_ans[i]==gbc_ans[i]):
        kfc.append(lgb_ans[i])
    elif (gbc_ans[i]==xgb_ans[i]):
        kfc.append(gbc_ans[i])
    else:
        kfc.append(gbc_ans[i])
        
print(len(kfc))
DF(kfc).to_csv('ans_fuck2.csv',index=False,header=False)

200
